### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
# Load data
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Encode categorical features
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Split data
X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [6]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [7]:
from scikeras.wrappers import BaseWrapper

# Create a Keras classifier using BaseWrapper
class MyKerasClassifier(BaseWrapper):
    def __init__(self, model=None, **kwargs):
        super().__init__(model=model, **kwargs)

In [8]:
## Create a Keras classifier
model=MyKerasClassifier(model=create_model, verbose=1)

In [11]:

# Define the grid search parameters
param_grid = {
    'model__neurons': [16, 32, 64, 128],
    'model__layers': [1, 2],
    'fit__epochs': [50, 100]
}

In [12]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50


d:\learning\Gen_AI\04_ANN_Churn_Modelling\venv\Lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


250/250 [==============================] - 1s 2ms/step - loss: 0.5232 - accuracy: 0.7764
Epoch 2/50
250/250 [==============================] - 0s 2ms/step - loss: 0.4337 - accuracy: 0.8163
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 0.4121 - accuracy: 0.8230
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3971 - accuracy: 0.8313
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3834 - accuracy: 0.8388
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3723 - accuracy: 0.8441
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3644 - accuracy: 0.8472
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3584 - accuracy: 0.8514
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3546 - accuracy: 0.8508
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3513 - accuracy: 0.8534
Epoch 11/5